 # Naive Bayes

 ## Importações

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
# otimizar processo de inicialização da tabela bayesiana
# implementar correção laplaciana

 ## Leitura de Dados

In [3]:
dados = pd.read_csv("dataset/diabetes_train.csv")

In [4]:
teste = pd.read_csv("dataset/diabetes_test.csv")

In [5]:
x_treinamento, x_teste = dados.iloc[:, :-1].values, teste.iloc[:, :-1].values

In [6]:
y_treinamento, y_teste = dados.iloc[:, -1].values, teste.iloc[:, -1].values

## Algoritmo (Sem Correção Laplaciana)

In [7]:
class NaiveBayes: 
    def __init__(self):
        self.tabela_bayesiana = {}
        self.categorias_target = None
        self.labels_features = None
        self.qtd_target = None
        self.qtd_features = None

    def inicializa_tabela_bayesiana(self, x):
        features = [[] for _ in range(self.qtd_features)]
        
        for i in range(len(x)):
            for j in range(self.qtd_features):
                features[j].append(x[i][j])

        for i in range(len(features)):
            features[i] = np.unique(features[i])

        for i in self.categorias_target:
            self.tabela_bayesiana[i] = {}
        
            for j, k in enumerate(features):
                self.tabela_bayesiana[i][j] = {}

                for l in k:
                    self.tabela_bayesiana[i][j][l] = 0

    def calcula_tabela_bayesiana(self, x, y):
        for i, j in enumerate(x):
            for k in range(len(j)):
                self.tabela_bayesiana[y[i]][k][j[k]] += 1
    
    def fit(self, x, y):
        counter_feature = np.unique(y, return_counts=True)
        self.qtd_target = counter_feature[1]
        self.categorias_target = counter_feature[0]
        self.qtd_features = int(x.size/len(x))

        self.inicializa_tabela_bayesiana(x)
        self.calcula_tabela_bayesiana(x, y)

        for i in self.categorias_target:
            for k in self.tabela_bayesiana[i]:
                for j in self.tabela_bayesiana[i][k]:
                    self.tabela_bayesiana[i][k][j] /= self.qtd_target[i]

    def calcula_probabilidade(self, x):
        p = []
        for i in self.categorias_target:
            produtorio = 1
            for j, k in enumerate(x):
                produtorio *= self.tabela_bayesiana[i][j][k]
            
            p.append(produtorio * self.qtd_target[i]) 

        for i, k in enumerate(p):
            if k == max(p):
                return self.categorias_target[i]

    def predict(self, x):
        return list(map(self.calcula_probabilidade, x))
    
    def confusion_matrix(self, previsoes, real):
        matriz_confusao = np.zeros((2, 2))

        for i, k in zip(previsoes, real):
            matriz_confusao[k][i] += 1

        return matriz_confusao
    
    def score(self, y_resultado, y_teste):
        matriz_confusao = self.confusion_matrix(y_resultado, y_teste)

        acuracia = np.sum(np.diagonal(matriz_confusao))/np.sum(matriz_confusao)
        precisao = (matriz_confusao[1][1]/(np.sum(matriz_confusao[:, 1])))
        revocacao = (matriz_confusao[1][1] / (np.sum(matriz_confusao[1][:])))
        f1_score = (2/((1/precisao) + 1/(revocacao)))

        print("=" * 10 + " Medições de Desempenho " + "=" * 10)
        print("Matriz de Confusão")
        print(f"{pd.DataFrame(matriz_confusao, columns=['0', '1'])}")
        print(f"\nAcurácia: {acuracia:.4f}")
        print(f"Precisão: {precisao:.4f}")
        print(f"Revocação: {revocacao:.4f}")
        print(f"F1-Score: {f1_score:.4f}")
        print("=" * 44)

In [8]:
naive_bayes = NaiveBayes()
naive_bayes.fit(x_treinamento, y_treinamento)
y_resultado = naive_bayes.predict(x_teste)

In [9]:
naive_bayes.score(y_resultado, y_teste)

========== Medições de Desempenho ==========
Matriz de Confusão
      0      1
0  74.0    7.0
1  17.0  110.0

Acurácia: 0.8846
Precisão: 0.9402
Revocação: 0.8661
F1-Score: 0.9016
